# Constants

In [1]:
import os, json
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")


project_endpoint = os.environ["AZURE_AIFSTDEASTUS_PROJECT_ENDPOINT"]
deployment_name =  "gpt-4.1" # os.environ["MODEL_DEPLOYMENT_NAME"] # 
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-04-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://basicfdrymlbe.services.ai.azure.com/api/projects/project>
azure-ai-projects library installed version: 1.0.0b12
azure-ai-agents library installed version: 1.1.0b3


# [Create AI Foundry Agent Client](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview)
**Note**: I could create the `project` client rather than the `agent` client, however this is easier to read.<br/>
Please consider that `project_client.agens == agents_client`

In [2]:
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

agents_client

# [MCP Agent](https://devblogs.microsoft.com/foundry/announcing-model-context-protocol-support-preview-in-azure-ai-foundry-agent-service/)
- [Connect to Model Context Protocol (MCP) Servers (Preview)](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)
- [How to use the Model Context Protocol (MCP) tool WITH http](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples)
- [AI Foundry Basic Agent Setup](https://github.com/azure-ai-foundry/foundry-samples/tree/main/samples/microsoft/infrastructure-setup/40-basic-agent-setup)
- [Announcing Model Context Protocol Support (preview) in Azure AI Foundry Agent Service](https://devblogs.microsoft.com/foundry/announcing-model-context-protocol-support-preview-in-azure-ai-foundry-agent-service/)
- Region: WestUS

aif02westusgrp
basicfdrydl5j
aif02basicproject01

In [3]:
# MCP Agent Instructions
agent_name = "MCP_learn_agent"

agent_instructions = "You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer " \
"queries about Microsoft Learn portal."

# Question
question = """
"Which Azure OpenAI models are being deprecated, and what can I use to replace them?
Please provide details for each model and version, highlighting the differences between versions.
Include deprecation dates and recommended alternative versions.
Build a final table.
"""

In [4]:
# MCP Tool definition

mcp_tool_definition = {
    "type": "mcp", #il tool è di tipo MCP
    "server_label": "microsoft_docs_search", #il nome univoco per MCP Server di MSLearn è "microsoft_docs_search"
    "server_url": "https://learn.microsoft.com/api/mcp", #l'URL del server MCP di MSLearn
    # "require_approval": "never" 
    "allowed_tools":["fetch_generic_documentation", "search_generic_code", "search_generic_documentation"]
}
mcp_tool_definition

{'type': 'mcp',
 'server_label': 'microsoft_docs_search',
 'server_url': 'https://learn.microsoft.com/api/mcp',
 'allowed_tools': ['fetch_generic_documentation',
  'search_generic_code',
  'search_generic_documentation']}

# Agent creation

In [5]:
agent_id = "" # ex: asst_j1qWBdGsjbK4hHcO0M0n3M5p. If provided, it will be loaded rather than created

if agent_id != "":
    agent = agents_client.get_agent(agent_id=agent_id)
else:
    agent = agents_client.create_agent(
        model=deployment_name,
        name=agent_name,
        instructions=agent_instructions,
        tools=[mcp_tool_definition], # leave this line commented to create a naked agent ;-)
        tool_resources=None,
        headers={"x-ms-enable-preview": "true"}
    )

agent.as_dict()

{'id': 'asst_YSuLw2c9qKXdNqkngW7fR2wq',
 'object': 'assistant',
 'created_at': 1751492292,
 'name': 'MCP_learn_agent',
 'description': None,
 'model': 'gpt-4.1',
 'instructions': 'You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer queries about Microsoft Learn portal.',
 'tools': [{'type': 'mcp',
   'server_label': 'microsoft_docs_search',
   'server_url': 'https://learn.microsoft.com/api/mcp',
   'allowed_tools': ['fetch_generic_documentation',
    'search_generic_code',
    'search_generic_documentation']}],
 'top_p': 1.0,
 'temperature': 1.0,
 'tool_resources': {},
 'metadata': {},
 'response_format': 'auto'}

# Create the thread and attach a new message to it

In [6]:
# Create a thread
thread = agents_client.threads.create()
print(f"Created thread: {thread}\n")

Created thread: {'id': 'thread_VjPAXB9kC9C2xKoR43Tj2qEc', 'object': 'thread', 'created_at': 1751492292, 'metadata': {}, 'tool_resources': {}}



In [7]:
# Add a user message to the thread
from azure.ai.agents.models import MessageRole

message = agents_client.messages.create(
    thread_id=thread.id, 
    role=MessageRole.USER, 
    content= question
)
print(f"Created message: {message}")

Created message: {'id': 'msg_PJan5C3TBSn6k85OzW3AkiOX', 'object': 'thread.message', 'created_at': 1751492293, 'assistant_id': None, 'thread_id': 'thread_VjPAXB9kC9C2xKoR43Tj2qEc', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n"Which Azure OpenAI models are being deprecated, and what can I use to replace them?\nPlease provide details for each model and version, highlighting the differences between versions.\nInclude deprecation dates and recommended alternative versions.\nBuild a final table.\n', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent syncrhonously

In [8]:
%%time

# Run the agent
run = agents_client.runs.create_and_process\
    (thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_o7ad2kBaxfZpEM7bntQVuBPF', 'object': 'thread.run', 'created_at': 1751492294, 'assistant_id': 'asst_YSuLw2c9qKXdNqkngW7fR2wq', 'thread_id': 'thread_VjPAXB9kC9C2xKoR43Tj2qEc', 'status': 'completed', 'started_at': 1751492294, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1751492303, 'required_action': None, 'last_error': None, 'model': 'gpt-4.1', 'instructions': 'You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer queries about Microsoft Learn portal.', 'tools': [{'type': 'mcp', 'server_label': 'microsoft_docs_search', 'server_url': 'https://learn.microsoft.com/api/mcp', 'allowed_tools': ['fetch_generic_documentation', 'search_generic_code', 'search_generic_documentation']}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': '

# Fetch messages from the thread after the agent run execution

In [9]:
from azure.ai.agents.models import MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, MessageTextFilePathAnnotation

image_files = []
annotations = []
citations = []

if run.status == 'completed':
    messages = agents_client.messages.list(thread_id=thread.id)
    messages_list = list(agents_client.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

Message 1 / CONTENT 1 (MessageTextContent) --> Text: 
"Which Azure OpenAI models are being deprecated, and what can I use to replace them?
Please provide details for each model and version, highlighting the differences between versions.
Include deprecation dates and recommended alternative versions.
Build a final table.


===== MESSAGE 2 =====

Message 2 / CONTENT 1 (MessageTextContent) --> Text: I’m currently unable to access the most recent documentation directly, but I can provide an overview based on existing knowledge about Azure OpenAI models deprecation, as well as general guidance. For the most accurate and detailed information, always refer to the official Azure OpenAI Service documentation.

### General Model Deprecations (as of early 2024)

#### GPT-3 and Early GPT-3.5 Models
- **Deprecated Models:**
  - `text-davinci-003`
  - `text-ada-001`, `text-babbage-001`, `text-curie-001`, etc.
- **Deprecation Dates:** 
  - Most legacy

# Print annotations from the messages

In [10]:
print (f"Nr. of file path annotations: {len(annotations)}\n")

i=0
for a in annotations:
    i += 1
    print(f"{i} - File annotation paths: {a.text}")
    file_name = a.text.split('/')[-1]
    file_id = a.file_path.file_id

    #agents_client.files.save(file_id=file_id, file_name=file_name)
    project_client.agents.files.save(file_id=file_id, file_name=file_name)
    print(f"\n>>> file <{file_id}> saved as <{file_name}>")
    
    # project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of file path annotations: 0



# Fetch citations from the messages

In [11]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

Nr. of file path citations: 0



# Retrieve and download eventual images

In [12]:
print (f"Nr. of image contents: {len(image_files)}\n")

i=0
# Generate an image file for the bar chart
for image_file in image_files:
    i += 1
    print(f"{i} - Image file id: {image_file}")
    file_name = f"{image_file}.png"
    project_client.agents.files.save(file_id=image_file, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of image contents: 0



# Collect all resources for this project

In [13]:
all_agents = list_all_agents(client=agents_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=agents_client)
print(all_threads["summary"])

all_files = list_all_files(client=agents_client)
print(all_files["summary"])

all_runs = list_all_runs(client=agents_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(agents_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(agents_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=agents_client)
print(all_vectorstores["summary"])

1 agents
1 threads
0 files
1 runs in 1 threads
0 vector stores


# Teardown for all resources

In [14]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]:
    i += 1
    project_client.agents.vector_stores.delete(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(client=agents_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [15]:
# delete all files

i=0
for file in all_files['content']:
    i += 1
    agents_client.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(agents_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



In [16]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    agents_client.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(agents_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_VjPAXB9kC9C2xKoR43Tj2qEc> has been deleted
Threads deleted: 1



In [17]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    agents_client.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=agents_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_YSuLw2c9qKXdNqkngW7fR2wq> has been deleted
Agents deleted: 1



# HIC SUNT LEONES